# Cat Face GAN model  

This is a project to test some of the concepts I have learnt for a Generative Adversial Network (GAN) and also to practise using Git. 

this is just a change line, nothing special.

In [5]:
dataset_dir = 'dataset'

In [6]:
import os 
from matplotlib.image import imread

In [29]:
imgpath='D:/data_science/GIT/Cat-Face-GAN-/dataset/'

In [30]:
from tensorflow.keras.preprocessing import image 


In [32]:
test = image.load_img(path=imgpath,
                      target_size=(64,64),color_mode='rgb')

PermissionError: [Errno 13] Permission denied: 'D:/data_science/GIT/Cat-Face-GAN-/dataset/'

In [33]:
cwd

NameError: name 'cwd' is not defined